# Functions for Preprosessing 
## for Metric Calculations with Data out of MongoDB

## Semantic Cosine Preprocessing-Execution

In [29]:
# 11) Semantic-Cosine–Preprocessing ausführen und in MongoDB ersetzen
from pymongo import MongoClient
from tqdm import tqdm

# 1) Verbindung & Collection
client = MongoClient("mongodb://localhost:27018/")
db     = client["transcriptions"]
coll   = db["transcripts_denis"]

# 2) Loop über alle Docs und Felder neu setzen
count = 0
for doc in tqdm(coll.find({"excludeGeneral": 1}), desc="Semantic Preprocessing"):
    raw_ref = doc.get("srcText", "")
    raw_hyp = doc.get("text", "")

    # Nutze die finale, korrekte Funktion
    sem_ref = preprocess_text_for_semantic_cosine(raw_ref)
    sem_hyp = preprocess_text_for_semantic_cosine(raw_hyp)

    coll.update_one(
        {"_id": doc["_id"]},
        {"$set": {
            "src_sem_denis": sem_ref,
            "text_sem_denis": sem_hyp
        }}
    )
    count += 1

print(f"{count} Dokumente mit Semantic-Preprocessing aktualisiert.")
client.close()


Semantic Preprocessing: 120it [00:00, 376.21it/s]

120 Dokumente mit Semantic-Preprocessing aktualisiert.


In [3]:
# 1) Imports & Client‐Setup CHECK
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

api_key = os.getenv("OPEN_API_KEY_STANIC")
if not api_key:
    raise ValueError("Bitte setze die Umgebungsvariable OPEN_API_KEY_STANIC.")
client = OpenAI(api_key=api_key)

# 2) Helper für Embeddings
def get_embedding(text: str, model: str="text-embedding-3-large") -> np.ndarray:
    resp = client.embeddings.create(model=model, input=[text])
    # im neuen Schema ist der Pfad: resp.data[0].embedding
    return np.array(resp.data[0].embedding)

# 3) Zwei kurze Beispiele
src_text = "Wir sind in der Wohnung. Paramedic Schmidt erklärt die Situation."
hyp_text = "wir sind in der wohnung paramedic schmidt erklaert situation"

# 4) Embeddings abrufen
emb_src = get_embedding(src_text)
emb_hyp = get_embedding(hyp_text)

# 5) Cosine Similarity berechnen
sim = float(cosine_similarity([emb_src], [emb_hyp])[0][0])

# 6) Ausgabe
print(f"Embedding-Dimension: {len(emb_src)}")   # sollte 3072 sein wenn text-embedding-3-large default dim
print(f"Cosine Similarity:   {sim:.4f}")



Embedding-Dimension: 3072
Cosine Similarity:   0.8885


In [8]:
from pymongo import MongoClient
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import os
import json  # neu
from datetime import datetime

# 1) OpenAI‐Client
#api_key   = os.getenv("OPEN_API_KEY_STANIC")
client_oa = OpenAI(api_key=api_key)

def batch_embeddings(texts: list[str], model: str="text-embedding-3-large") -> list[np.ndarray]:
    resp = client_oa.embeddings.create(model=model, input=texts)
    return [np.array(d.embedding) for d in resp.data]

# 2) Mongo öffnen und bereits vorprocessed Docs holen
mongo      = MongoClient("mongodb://localhost:27018/")
coll       = mongo["transcriptions"]["transcripts_denis"]
docs       = list(coll.find({
    "excludeGeneral": 0,
    "src_sem_denis":  {"$exists": True},
    "text_sem_denis": {"$exists": True}
}))

# 2a) JSONL-File öffnen (neu)
timestamp = datetime.now().strftime("%d%m%y_%H%M")
out_dir   = f"sem_embeddings_{timestamp}"
os.makedirs(out_dir, exist_ok=True)
jsonl_f   = open(f"{out_dir}/embeddings.jsonl", "w", encoding="utf-8")

# 3) Batch-Größe
BATCH_SIZE = 40

# 4) In Batches durchlaufen
for i in tqdm(range(0, len(docs), BATCH_SIZE), 
              total=len(docs)//BATCH_SIZE+1, desc="Batches"):
    batch     = docs[i : i + BATCH_SIZE]
    src_texts = [d["src_sem_denis"]  for d in batch]
    hyp_texts = [d["text_sem_denis"] for d in batch]

    emb_srcs = batch_embeddings(src_texts)
    emb_hyps = batch_embeddings(hyp_texts)

    for doc, e_src, e_hyp in zip(batch, emb_srcs, emb_hyps):
        sem_cos = float(cosine_similarity([e_src], [e_hyp])[0][0])

        # 4a) Mongo-Update
        coll.update_one(
            {"_id": doc["_id"]},
            {"$set": {"sem_cos_denis": sem_cos}}
        )

        # 4b) JSONL schreiben (neu)
        jsonl_f.write(json.dumps({
            "_id":           str(doc["_id"]),
            "technology":    doc.get("technology"),
            "model":         doc.get("model"),
            "fileName":      doc.get("fileName"),
            "convoID":       doc.get("convoID"),
            "ambientVariant":doc.get("ambientVariant"),
            "processedVolume":doc.get("processedVolume"),
            "sem_cos_denis": sem_cos,
            "src_embeddings": e_src.tolist(),
            "text_embeddings":e_hyp.tolist()
        }, ensure_ascii=False) + "\n")

# 5) Aufräumen
jsonl_f.close()
mongo.close()


Batches: 100%|█████████████████████████████▉| 297/298 [13:28<00:02,  2.72s/it]


## Check if the the embedding and cos_sim worked

In [10]:
import os
import json
from glob import glob

# 1) Finde das neueste JSONL-File
jsonl_files = sorted(glob("sem_embeddings_*/embeddings.jsonl"))
if not jsonl_files:
    raise FileNotFoundError("Keine embeddings.jsonl Datei gefunden.")
jsonl_path = jsonl_files[-1]
print(f"Überprüfe Datei: {jsonl_path}\n")

# 2) Initialisiere Zähler und Sammler
total_lines = 0
missing_entries = []
dim_mismatches = []

# 3) Durchlaufe jede Zeile und prüfe Felder und Dimensions
with open(jsonl_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f, start=1):
        rec = json.loads(line)
        total_lines += 1
        
        # Prüfe auf notwendige Keys
        for key in ["_id", "technology", "model", "fileName", "convoID", 
                    "ambientVariant", "processedVolume", "sem_cos_denis", 
                    "src_embeddings", "text_embeddings"]:
            if key not in rec:
                missing_entries.append((idx, key))
        
        # Prüfe Embedding-Dimensionen
        if "src_embeddings" in rec and len(rec["src_embeddings"]) != 3072:
            dim_mismatches.append(("src", idx, len(rec["src_embeddings"])))
        if "text_embeddings" in rec and len(rec["text_embeddings"]) != 3072:
            dim_mismatches.append(("text", idx, len(rec["text_embeddings"])))

# 4) Ergebnisse ausgeben
print(f"Gesamtanzahl Zeilen: {total_lines}")
if not missing_entries:
    print("✅ Alle Einträge enthalten alle benötigten Keys.")
else:
    print("❌ Fehlende Keys in folgenden Zeilen:")
    for line, key in missing_entries:
        print(f"  Zeile {line}: fehlt Key '{key}'")

if not dim_mismatches:
    print("✅ Alle Embeddings haben die korrekte Dimension (3072).")
else:
    print("❌ Dimensionsabweichungen gefunden:")
    for which, line, dim in dim_mismatches:
        print(f"  Zeile {line}: {which}_embeddings Länge = {dim}")


Überprüfe Datei: sem_embeddings_040725_1817/embeddings.jsonl

Gesamtanzahl Zeilen: 11880
✅ Alle Einträge enthalten alle benötigten Keys.
✅ Alle Embeddings haben die korrekte Dimension (1536).


In [11]:
import os
import json
import numpy as np
import pandas as pd

# Pfad zur JSONL-Datei – ggf. anpassen
jsonl_path = "sem_embeddings_040725_1817/embeddings.jsonl"

# 1) Einträge prüfen auf fehlende bzw. NaN‐Werte
missing = []
sem_cos = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f, 1):
        rec = json.loads(line)
        val = rec.get("sem_cos_denis")
        if val is None or (isinstance(val, float) and np.isnan(val)):
            missing.append(i)
        else:
            sem_cos.append(val)

print(f"📄 Zeilen insgesamt: {i}")
print(f"❌ Fehlende/NaN‐Einträge in Zeilen: {missing[:10]}{'...' if len(missing)>10 else ''}")
print(f"✅ Vollständige Einträge: {len(sem_cos)}")

# 2) Deskriptive Statistik
stats = {
    "Mean":    np.mean(sem_cos),
    "Median":  np.median(sem_cos),
    "StdDev":  np.std(sem_cos, ddof=1),
    "Min":     np.min(sem_cos),
    "Max":     np.max(sem_cos)
}

# 3) Ausgabe als DataFrame
df = pd.DataFrame.from_dict(stats, orient="index", columns=["Value"])
display(df)


📄 Zeilen insgesamt: 11880
❌ Fehlende/NaN‐Einträge in Zeilen: []
✅ Vollständige Einträge: 11880


,Value
Mean,0.958542
Median,0.974911
StdDev,0.041282
Min,0.646001
Max,0.998488


In [13]:
# %%  
# 1) Imports
import pandas as pd
import os

# 2) Pfad zur JSONL-Datei – passe das Datums-/Zeit-Stempel-Verzeichnis an, falls nötig
jsonl_dir  = sorted([d for d in os.listdir() if d.startswith("sem_embeddings_")])[-1]
jsonl_path = os.path.join(jsonl_dir, "embeddings.jsonl")

# 3) JSONL als DataFrame laden
df = pd.read_json(jsonl_path, lines=True)

# 4) Nur die gewünschten Spalten auswählen
df_csv = df[[
    "convoID",
    "ambientVariant",
    "processedVolume",
    "technology",
    "model",
    "fileName",
    "sem_cos_denis"
]]

# 5) CSV speichern im Notebook-Ordner
out_csv = "sem_cos_results.csv"
df_csv.to_csv(out_csv, index=False, encoding="utf-8")

print(f"✅ CSV geschrieben: {out_csv}")


✅ CSV geschrieben: sem_cos_results.csv


In [13]:
# 1) Imports & MongoDB-Verbindung
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Verbinden und Collection auswählen
client = MongoClient("mongodb://localhost:27018/")
db     = client["transcriptions"]
coll   = db["transcripts_denis"]

# Alle Dokumente (ohne manuell ausgesperrte)
docs = list(coll.find({"excludeGeneral": 0}))


In [14]:
# 2) TF-IDF auf gesamten Korpus fitten
# Wir brauchen globale IDF-Werte, also fitten wir auf ALLE Vorverarbeitungen:
lex_hyps = [doc["text_lex_denis"] for doc in docs]
lex_refs = [doc["src_lex_denis"] for doc in docs]
corpus   = lex_hyps + lex_refs

vectorizer    = TfidfVectorizer()
tfidf_matrix  = vectorizer.fit_transform(corpus)


In [15]:
# 3) Paarweise Cosine-Ähnlichkeit berechnen und in DataFrame packen
rows = []
N = len(docs)
for i, doc in enumerate(docs):
    hyp_vec = tfidf_matrix[i]
    ref_vec = tfidf_matrix[i + N]
    sim     = cosine_similarity(hyp_vec, ref_vec)[0][0]

    rows.append({
        "convoID":           doc.get("convoID"),
        "ambientVariant":    doc.get("ambientVariant"),
        "processedVolume":   doc.get("processedVolume"),
        "technology":        doc.get("technology"),
        "model":             doc.get("model"),
        "lex_cosine_sim":    sim
    })

df_lex = pd.DataFrame(rows)
df_lex.to_csv("lexical_cosine_scores_full.csv", index=False, encoding="utf-8-sig")
print("Lexical Cosine Similarity scores saved to lexical_cosine_scores_full.csv")


Lexical Cosine Similarity scores saved to lexical_cosine_scores_full.csv
